In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
import scipy.signal
import matplotlib.ticker as plticker
import time
from mpl_toolkits.axes_grid1 import make_axes_locatable
from tqdm import tqdm
from collections import defaultdict
from typing import Optional
from scipy.integrate import solve_ivp
from scipy.fftpack import diff as psdiff
import torch
import pytorch_lightning as pl

import os, sys
sys.path.append(os.path.join(os.getcwd(), '../sympde'))

from data.dataset import PDEDataset, PDEDataModule
from data.utils import d_to_LT

from model.setup import setup_model
from model.networks.fno import FNO1d
from model.learner import Learner

from run import parse_options

In [2]:
args = parse_options(notebook=True)

In [3]:
dataset = PDEDataset(pde_name = args.pde_name, data_dir = args.data_dir,)
us, dx, dt = dataset[0]

In [4]:
datamodule = PDEDataModule(pde_name = args.pde_name, data_dir = args.data_dir, batch_size = args.batch_size, num_workers = args.num_workers)

In [5]:
datamodule.setup()

In [6]:
batch = next(iter(datamodule.train_dataloader()))
batch

[tensor([[[0.3008, 0.3036, 0.3048,  ..., 0.2833, 0.2907, 0.2965],
          [0.2983, 0.3010, 0.3023,  ..., 0.2807, 0.2881, 0.2940],
          [0.2958, 0.2985, 0.2998,  ..., 0.2782, 0.2856, 0.2914],
          ...,
          [0.2142, 0.2175, 0.2195,  ..., 0.1963, 0.2036, 0.2095],
          [0.2120, 0.2154, 0.2174,  ..., 0.1942, 0.2014, 0.2074],
          [0.2099, 0.2132, 0.2153,  ..., 0.1920, 0.1993, 0.2052]]]),
 tensor([0.2500]),
 tensor([1.0256])]

In [7]:
model = setup_model(args)
y_hat, y = model(batch)

In [8]:
model.step(batch)

/Users/elias/anaconda3/envs/sympde/lib/python3.10/site-packages/pytorch_lightning/core/module.py:420: You are trying to `self.log()` but the `self.trainer` reference is not registered on the model yet. This is most likely because the model hasn't been passed to the `Trainer`


(tensor([[[0.0653, 0.0668, 0.0675,  ..., 0.0566, 0.0601, 0.0630],
          [0.0443, 0.0456, 0.0462,  ..., 0.0372, 0.0401, 0.0425],
          [0.0591, 0.0605, 0.0613,  ..., 0.0508, 0.0542, 0.0570],
          [0.0503, 0.0517, 0.0524,  ..., 0.0427, 0.0458, 0.0484],
          [0.0436, 0.0449, 0.0455,  ..., 0.0366, 0.0394, 0.0418]]],
        grad_fn=<MseLossBackward0>),
 tensor([[[0.0205, 0.0205, 0.0205,  ..., 0.0205, 0.0205, 0.0205],
          [0.0630, 0.0630, 0.0630,  ..., 0.0630, 0.0630, 0.0630],
          [0.0281, 0.0281, 0.0281,  ..., 0.0281, 0.0281, 0.0281],
          [0.0444, 0.0444, 0.0444,  ..., 0.0444, 0.0444, 0.0444],
          [0.0575, 0.0575, 0.0575,  ..., 0.0575, 0.0575, 0.0575]]],
        grad_fn=<PermuteBackward0>))

In [9]:
trainer = pl.Trainer(
    logger=pl.loggers.TensorBoardLogger(
        args.log_dir, name=args.net, version=args.version
    ),
    max_epochs=args.max_epochs,
    log_every_n_steps=1,
    accelerator="gpu" if torch.cuda.is_available() else "cpu",
    callbacks = [
        pl.callbacks.TQDMProgressBar(refresh_rate=1000),
    ],
    deterministic=True
)

GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/Users/elias/anaconda3/envs/sympde/lib/python3.10/site-packages/pytorch_lightning/trainer/setup.py:187: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.


In [10]:
trainer.fit(model, datamodule=datamodule)

AttributeError: 'PDEDataModule' object has no attribute 'prepare_data_per_node'